In [1]:
def create_data_frame(data_src): 
    import pandas as pd
    
    data_labels = ['Order_ID', 'Customer_ID', 'Mean_Order_Amt', 'Treated',
                   'Treatment_Modifier', 'Retailer_ID', 'Retailer_Scalar',
                   'Dow_Rand', 'DOW', 'Noise', 'Order_Amt']
     
    pa_retailer_means = pd.DataFrame(data_src.groupby(['Retailer_ID'])['Order_Amt'].mean())
    pa_retailer_means.reset_index(inplace=True)
    pa_retailer_means.columns = ['Retailer_ID', 'Mean_Retailer_Order_Amt']
    ###############################################################################
    pa_dow_means = pd.DataFrame(data_src.groupby(['Dow_Rand'])['Order_Amt'].mean())
    pa_dow_means.reset_index(inplace = True)
    pa_dow_means.columns = ['Dow_Rand', 'Mean_DOW_Order_Amt']
    ###############################################################################
    analysis_df = pd.merge(data_src, pa_retailer_means, on = 'Retailer_ID', how = 'left')
    analysis_df = pd.merge(analysis_df, pa_dow_means, on = 'Dow_Rand', how = 'left')
    ###############################################################################
    analysis_df = analysis_df[['Order_Amt', 'Treated', 'Customer_ID', 'Mean_Order_Amt', 
                               'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
    return analysis_df

In [2]:
def assess_point(point_n, num_sim, threshold, data):
    from statistics import mean
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    
    pvalues  = []
    detected = []
    
#     print("Estimating power using n = {} with a p-value threshold of {}.".format(point_n, threshold))
    for i in list(range(0, num_sim, 1)):
        working_df = data.sample(point_n, replace=False)
        X = working_df[['Treated', 'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
        X = sm.add_constant(X)
        Y = working_df[['Order_Amt']]
        model = sm.OLS(Y, X).fit(method='pinv').get_robustcov_results(
                'cluster', groups = working_df['Customer_ID'], 
                use_correction=True, df_correction=True)
        pvalues.append(model.pvalues[1])
#         print(model.summary())
        if model.pvalues[1] < threshold:
            detected.append(1)
        else:
            detected.append(0)
    power = mean(detected)
#     print("Power was estimated as {}.".format(power))
    return power, pvalues

In [3]:
def create_rec(points, sub_iterations, threshold, desired_power, data):
    import numpy as np
    from scipy.optimize import curve_fit
    import time    

    start = time.time()
    master_results = []
    for i in points:
        power, pvalues = assess_point(i, sub_iterations, 0.05, data)
        master_results.append(power)
        
    def exp_func(x, a, b, c):
        return a * np.log(b * x) + c

    eta = []
    for i in points:
        eta.append(i)
    eta = np.asarray(eta)
 #   print(eta)
    cdf = []
    for i in master_results:
        cdf.append(i)
    cdf = np.asarray(cdf)
  #  print(cdf)

    popt, pcov = curve_fit(exp_func, eta, cdf)
    
    recommended_n = int(np.exp((desired_power - popt[2])/popt[0])/popt[1])    
    end = time.time()
    time_req = (end - start)/60.0
    return recommended_n, master_results, time_req

In [4]:
def verify_sample_size_est(sample_size, alpha, verify_n_times, data):

    import statsmodels.api as sm
    import pandas as pd
    
    SAMPLE_SIZE = sample_size
    VERIFICATION_ITERATIONS = verify_n_times
    ALPHA = alpha

    
    data_labels = ['Order_ID', 'Customer_ID', 'Mean_Order_Amt', 'Treated',
                   'Treatment_Modifier', 'Retailer_ID', 'Retailer_Scalar',
                   'Dow_Rand', 'DOW', 'Noise', 'Order_Amt']
    
    i = 0
    pvals  = []
    r_sqr  = []
    cond_n = []
    while i < VERIFICATION_ITERATIONS:
        try:
            analysis_df = data.sample(sample_size, replace=False)
        except:
            print("Failed to collect subsample of size {}. Reattempting".format())
            analysis_df = data.sample(sample_size, replace=False)
            
        X = analysis_df[['Treated', 'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
        X = sm.add_constant(X)
        Y = analysis_df[['Order_Amt']]
        model = sm.OLS(Y.astype(float), X.astype(float)).fit(method='pinv'). \
                       get_robustcov_results('cluster', groups = analysis_df['Customer_ID'], 
                       use_correction=True, df_correction=True)
        if model.pvalues[1] < ALPHA: 
            pvals.append(1)
        else:
            pvals.append(0)  

        r_sqr.append(model.rsquared_adj)
        cond_n.append(model.condition_number)
        i += 1
        if i % int((VERIFICATION_ITERATIONS)/10.0) == 0:
            completion = str(round((i/VERIFICATION_ITERATIONS)*100, 2))+'%'
            print(completion + ' complete.')
            
    # ----- Exit inner loop     
    x = ['Treated', 'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']
    str_out = 'Order_Amt =' 
    d = 0
    for i in x:
        if d < 1:
            k = " '" + i + "'"
        else:
            k = " + '" + i + "'"
        str_out += k
        d += 1    
    
    actual_power = sum(pvals)/len(pvals)  
   # print(pvals)
    mean_r_sqr   = sum(r_sqr)/len(r_sqr)   
    mean_cond_n  = sum(cond_n)/len(cond_n)  
    #print("Actual power was estimated at {}.".format(actual_power))
    return actual_power, mean_r_sqr, mean_cond_n, str_out

In [ ]:
def create_df(points, meta_iterations, sub_iterations, threshold, desired_power,verification_n):
    import pandas as pd
    
    dir_name = '/home/bknight/Documents/Power_Analysis_Techniques/variance_of_residuals/residual_dfs/'
    file_A = 'part_II_df_mde_0_01_n_100000_a.csv'
    file_B = 'part_II_df_mde_0_01_n_100000_b.csv'
    data_A = pd.read_csv(dir_name + file_A)
    data_B = pd.read_csv(dir_name + file_B)
    data_a = create_data_frame(data_A)
    data_b = create_data_frame(data_B)
    ##########################################################################################################

    recommendations       = []
    time_required         = []
    desired_p_list        = []
    actual_power_list     = []
    mean_r_sqr_list       = []
    mean_cond_n_list      = []
    str_out_list          = []
    n_points              = []
    point_list            = []
    simulations_per_point = []
    verification_steps    = []


    for i in list(range(0, meta_iterations, 1)):
        recommended_n, results, time_req = create_rec(points, sub_iterations, 
                                                      threshold, desired_power, data_a)
        actual_power, mean_r_sqr, mean_cond_n, str_out = \
                      verify_sample_size_est(recommended_n, threshold, verification_n, data_b)

        recommendations.append(recommended_n)
        time_required.append(time_req)
        desired_p_list.append(desired_power) 
        actual_power_list.append(actual_power)
        mean_r_sqr_list.append(mean_r_sqr)
        mean_cond_n_list.append(mean_cond_n)
        str_out_list.append(str_out)
        n_points.append(len(points))
        point_list.append(points)
        simulations_per_point.append(sub_iterations)
        verification_steps.append(verification_n)

    df = pd.DataFrame([recommendations, time_required, desired_p_list, actual_power_list, 
                       mean_r_sqr_list, mean_cond_n_list, str_out_list, n_points, 
                       point_list, simulations_per_point, verification_steps]).T
    df.columns = ['Recommended N', 'Time Required', 'Desired Power', 'Actual Power',
                  'Avg. R-Squared', 'Mean Cond. Number', 'Model Specification', '# Points', 
                  'Points', 'Simulations per Point', 'Verification Simulations']
    df.to_csv("Model_II_{}_Points_{}_Iterations.csv".format(len(points), sub_iterations))
    return df

In [ ]:
meta_iterations = 100

frameworks = \
[
(100, [30000, 40000, 50000]),
(100, [20000, 30000, 40000, 50000, 60000]),
(100, [10000, 20000, 30000, 40000, 50000, 60000, 70000]),
(200, [30000, 40000, 50000]),
(200, [20000, 30000, 40000, 50000, 60000]),
(200, [10000, 20000, 30000, 40000, 50000, 60000, 70000]),
(300, [30000, 40000, 50000]),
(300, [20000, 30000, 40000, 50000, 60000]),
(300, [10000, 20000, 30000, 40000, 50000, 60000, 70000])  
]

for i in list(range(0, len(frameworks), 1)):
    df = create_df(frameworks[i][1], meta_iterations, frameworks[i][0], 0.05, 0.8, 500)
    print("Framework {} completed.".format(str(i)))

/home/bknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]
/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


...

10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.


/home/bknight/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Framework 6 completed.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
10.0% compl